# Chinese Word Segmentation

## Download the data and Prepare training/test data

In [1]:
import requests

remote_url = "https://raw.githubusercontent.com/hhhuang/nlp2019fall/master/word_segmentation/"

r = requests.get(remote_url + "data/as_training.utf8", allow_redirects=True)
open('as_training.utf8', 'wb').write(r.content)

r = requests.get(remote_url + "data/as_testing_gold.utf8", allow_redirects=True)
open('as_testing_gold.utf8', 'wb').write(r.content)


921600

In [2]:
raw_train = []
raw_test = []
with open("as_training.utf8", encoding="utf8") as fin:
    for line in fin:
        raw_train.append(line.strip().split("　"))   # It is a full white space

with open("as_testing_gold.utf8", encoding="utf8") as fin:
    for line in fin:
        raw_test.append(line.strip().split("　"))   # It is a full white space

print("Number of sentences in the training data: %d" % len(raw_train))
print("Number of sentences in the test data: %d" % len(raw_test))

Number of sentences in the training data: 708953
Number of sentences in the test data: 14113


## Use jieba

In [3]:
# ! pip install jieba

In [3]:
import jieba

print(list(jieba.cut("".join(raw_test[0]))))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.284 seconds.
Prefix dict has been built successfully.


['許多', '社區長', '青學苑', '多', '開設', '有書法', '、', '插花', '、', '土風', '舞班', '，']


## Perform jieba on Simplified Chinese and restore the segmetnation to the original input. 

In [5]:
# ! pip install hanziconv

In [4]:
from hanziconv.hanziconv import HanziConv

print(list(jieba.cut(HanziConv.toSimplified("".join(raw_test[0])))))

['许多', '社区', '长青', '学苑', '多', '开设', '有', '书法', '、', '插花', '、', '土风舞', '班', '，']


In [5]:
def restore(text, toks):
    results = []
    offset = 0
    for tok in toks:
        results.append(text[offset:offset + len(tok)])
        offset += len(tok)
    return results

text = "".join(raw_test[0])
print(restore(text, list(jieba.cut(HanziConv.toSimplified(text)))))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞', '班', '，']


## Build Your Own Chinese Word Segmenter

Prepare training instances for sequence labeling by converting a list of words to a sequence of tags

Create a CRF model for word segmentation

In [8]:
# ! pip install sklearn-crfsuite

##Assignment II

Modify the following block ONLY for Assignment II

In [28]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics


def words_to_tags(words):
    tags = []
    for word in words:
        if len(word) == 1:
            tags.append('S')
        else:
            for i in range(len(word)):
                if i == 0:
                    tags.append('L')
                elif i == len(word) - 1:
                    tags.append('R')
                else:
                    tags.append('M')
    return tags
    
train_X = []
train_Y = []

test_X = []
test_Y = []

for sent in raw_train:
    train_X.append(list("".join(sent)))  # Make the unsegmented sentence as a sequence of characters
    train_Y.append(words_to_tags(sent))
    
for sent in raw_test:
    test_X.append(list("".join(sent)))  # Make the unsegmented sentence
    test_Y.append(words_to_tags(sent))
    
print(test_X[0])
print(test_Y[0])

dictionary = {}

def extract_sent_features(x):
    sent_features = []
    for i in range(len(x)):
        sent_features.append(extract_char_features(x, i))
    return sent_features

def extract_char_features(sent, position):
    char_features = {}
    for i in range(-3, 4):
        if len(sent) > position + i >= 0:
            char_features['char_at_%d' % i] = sent[position + i]
            if i != 0 and len(sent) > position + i - 1 >= 0:
                char_features['bigram_at_%d' % i] = sent[position + i - 1] + sent[position + i]
            
            # dict
            for dict_name, dict_values in dictionary.items():
                if sent[position + i] in dict_values:
                    char_features['%s_at_%d' % (dict_name, i)] = dict_values[sent[position + i]]
    return char_features



crf_tagger = sklearn_crfsuite.CRF(algorithm='lbfgs', min_freq=10, max_iterations=400, verbose=True)

feature_X = []
for x in train_X:
    feature_X.append(extract_sent_features(x))


try:
    crf_tagger.fit(feature_X, train_Y)
except AttributeError:
    pass

print(len(dictionary))

['許', '多', '社', '區', '長', '青', '學', '苑', '多', '開', '設', '有', '書', '法', '、', '插', '花', '、', '土', '風', '舞', '班', '，']
['B', 'E', 'B', 'E', 'B', 'E', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'E', 'S', 'B', 'M', 'M', 'E', 'S']


loading training data to CRFsuite: 100%|██████████| 708953/708953 [01:06<00:00, 10595.23it/s]



Feature generation
type: CRF1d
feature.minfreq: 10.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 663905
Seconds required: 21.632

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 400
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=4.52  loss=9522018.67 active=661577 feature_norm=1.00
Iter 2   time=2.25  loss=5946874.64 active=663905 feature_norm=5.09
Iter 3   time=4.49  loss=5615777.88 active=663905 feature_norm=5.61
Iter 4   time=2.25  loss=5296766.32 active=663905 feature_norm=6.41
Iter 5   time=2.27  loss=3875235.83 active=663905 feature_norm=12.40
Iter 6   time=2.25  loss=3588306.46 active=663905 feature_norm=14.71
Iter 7   time=2.26  loss=3352623.59 active=663905 feature_norm=14.62
Iter 8   time=2.26  loss=3178033.82 active=663905 feature_norm=16.42
Iter 9   time=2.26  loss=2965406.06 active=66390

In [9]:
import itertools
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics

def words_to_tags(words):
    tags = []
    for word in words:
        if len(word) == 1:
            tags.append('S')
        else:
            for i in range(len(word)):
                if i == 0:
                    tags.append('L')
                elif i == len(word) - 1:
                    tags.append('R')
                else:
                    tags.append('M')
    return tags
    
train_X = []
train_Y = []

test_X = []
test_Y = []

for sent in raw_train:
    train_X.append(list("".join(sent)))  # Make the unsegmented sentence as a sequence of characters
    train_Y.append(words_to_tags(sent))
    
for sent in raw_test:
    test_X.append(list("".join(sent)))  # Make the unsegmented sentence
    test_Y.append(words_to_tags(sent))

dictionary = {}

def extract_sent_features(x):
    sent_features = []
    for i in range(len(x)):
        sent_features.append(extract_char_features(x, i))
    return sent_features

def extract_char_features(sent, position):
    char_features = {}
    for i in range(-3, 4):
        if len(sent) > position + i >= 0:
            char_features['char_at_%d' % i] = sent[position + i]
            if i != 0 and len(sent) > position + i - 1 >= 0:
                char_features['bigram_at_%d' % i] = sent[position + i - 1] + sent[position + i]
            
            # dict
            for dict_name, dict_values in dictionary.items():
                if sent[position + i] in dict_values:
                    char_features['%s_at_%d' % (dict_name, i)] = dict_values[sent[position + i]]
    return char_features


crf_tagger = sklearn_crfsuite.CRF(algorithm='lbfgs', min_freq=10, max_iterations=1000, verbose=True)

feature_X = []
for x in train_X:
    feature_X.append(extract_sent_features(x))


try:
    crf_tagger.fit(feature_X, train_Y)
    # crf_tagger.fit(new_train_X, new_train_Y)
except AttributeError:
    pass


loading training data to CRFsuite: 100%|██████████| 708953/708953 [01:05<00:00, 10805.96it/s]



Feature generation
type: CRF1d
feature.minfreq: 10.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 663905
Seconds required: 20.425

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 1000
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=4.74  loss=9522018.67 active=661577 feature_norm=1.00
Iter 2   time=2.37  loss=5946874.64 active=663905 feature_norm=5.09
Iter 3   time=4.75  loss=5615777.88 active=663905 feature_norm=5.61
Iter 4   time=2.38  loss=5296766.32 active=663905 feature_norm=6.41
Iter 5   time=2.38  loss=3875235.83 active=663905 feature_norm=12.40
Iter 6   time=2.38  loss=3588306.46 active=663905 feature_norm=14.71
Iter 7   time=2.39  loss=3352623.59 active=663905 feature_norm=14.62
Iter 8   time=2.37  loss=3178033.82 active=663905 feature_norm=16.42
Iter 9   time=2.40  loss=2965406.06 active=6639

In [10]:
def segment(sent):
    tags = crf_tagger.predict_single(extract_sent_features(list(sent)))
    tokens = []
    tok = ""
    for ch, tag in zip(list(sent), tags):
        if tag in ['S', 'L'] and tok != "":
            tokens.append(tok)
            tok = ""
        tok += ch
    if tok:
        tokens.append(tok)
    return tokens
            
print(segment("法國總統馬克宏已到現場勘災，初步傳出火警可能與目前聖母院的維修工程有關。"))

['法國', '總統', '馬克宏', '已', '到', '現場', '勘災', '，', '初步', '傳出', '火警', '可能', '與', '目前', '聖母院', '的', '維修', '工程', '有關', '。']


## Evaluation
Scorer for CWS

In [7]:
def compare(actual_toks, pred_toks):
    i = 0
    j = 0
    p = 0
    q = 0
    tp = 0
    fp = 0
    while i < len(actual_toks) and j < len(pred_toks):
        if p == q:
            if actual_toks[i] == pred_toks[j]:
                tp += 1
            else:
                fp += 1
            p += len(actual_toks[i])
            q += len(pred_toks[j])
            i += 1
            j += 1
        elif p < q:
            p += len(actual_toks[i])
            i += 1
        else:
            fp += 1
            q += len(pred_toks[j])
            j += 1
    return tp, fp, len(actual_toks)
    
def score(actual_sents, pred_sents):
    tp = 0
    fp = 0
    total = 0
    for actual_toks, pred_toks in zip(actual_sents, pred_sents):
        tp_, fp_, total_ = compare(actual_toks, pred_toks)
        tp += tp_
        fp += fp_
        total += total_
    recall = float(tp) / total
    precision = float(tp) / (tp + fp)
    f1 = 2.0 * recall * precision / (recall + precision)
    return recall, precision, f1

Testing

In [11]:
pred = []
actual = []
for sent in raw_test:
    pred.append(segment("".join(sent)))
    actual.append(sent)
print(actual[0])
print(pred[0])

print(score(actual, pred))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區', '長青', '學苑', '多', '開', '設有', '書法', '、', '插花', '、', '土風舞班', '，']
(0.9419975832326347, 0.9315789907447852, 0.9367593191122604)


In [26]:
pred = []
actual = []
fout = open("jieba.out", "w")
for sent in raw_test:
    text = "".join(sent)
    r = list(jieba.cut(HanziConv.toSimplified(text)))
    r = restore(text, r)
    fout.write(" ".join(r) + "\n")
    pred.append(r)
    actual.append(sent)
print(actual[0])
print(pred[0])

print(score(actual, pred))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞', '班', '，']
(0.8145756073169715, 0.8291133335029816, 0.8217801804225554)


In [14]:
# ! pip install simpletransformers

In [15]:
import pandas as pd

def prepare_data(X, Y, limit=None):
    data = []
    for sid, (x, y) in enumerate(zip(X, Y)):
        for x_tok, y_tok in zip(x, y):
            data.append([sid, x_tok, y_tok])
        if limit and sid >= limit:
            break
    return data

train_data = prepare_data(train_X, train_Y, 3000)
eval_data = prepare_data(test_X, test_Y, 100)
print(train_data[:10])
print(eval_data[:10])

train_data = pd.DataFrame(train_data, columns=["sentence_id", "words", "labels"])
eval_data = pd.DataFrame(eval_data, columns=["sentence_id", "words", "labels"])

[[0, '時', 'L'], [0, '間', 'R'], [0, '：', 'S'], [1, '三', 'L'], [1, '月', 'R'], [1, '十', 'L'], [1, '日', 'R'], [1, '（', 'S'], [1, '星', 'L'], [1, '期', 'M']]
[[0, '許', 'L'], [0, '多', 'R'], [0, '社', 'L'], [0, '區', 'R'], [0, '長', 'L'], [0, '青', 'R'], [0, '學', 'L'], [0, '苑', 'R'], [0, '多', 'S'], [0, '開', 'L']]


In [16]:
from simpletransformers.ner import NERModel, NERArgs

model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.labels_list = ["L", "M", "R", "S"]
model_args.overwrite_output_dir = True

model = NERModel("bert", "bert-base-chinese", args=model_args)
model.train_model(train_data, eval_data=eval_data)
result, model_outputs, preds_list = model.eval_model(eval_data)

/home/ttsai/anaconda3/envs/pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from

In [17]:
predictions, raw_outputs = model.predict([" ".join(list("今天天氣晴朗"))])
print(predictions)
predictions, raw_outputs = model.predict([" ".join(list("天朗氣清、惠風和暢"))])
print(predictions)

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 172.93it/s]

[[{'今': 'L'}, {'天': 'R'}, {'天': 'L'}, {'氣': 'R'}, {'晴': 'L'}, {'朗': 'R'}]]



Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 175.16it/s]

[[{'天': 'L'}, {'朗': 'M'}, {'氣': 'M'}, {'清': 'R'}, {'、': 'S'}, {'惠': 'L'}, {'風': 'M'}, {'和': 'R'}, {'暢': 'S'}]]


In [18]:
def bert_segment(sents):
    for i in range(len(sents)):
        sents[i] = " ".join(list(sents[i]))
    
    predictions, _ = model.predict(sents)
    sent_tokens = []
    for prediction in predictions:
        tokens = []
        tok = ""
        for kv in prediction:
            for ch, tag in kv.items():
                if tag in ['S', 'L'] and tok != "":
                    tokens.append(tok)
                    tok = ""
                tok += ch
        if tok:
            tokens.append(tok)
        sent_tokens.append(tokens)
    return sent_tokens
            
print(bert_segment(["法國總統馬克宏已到現場勘災，初步傳出火警可能與目前聖母院的維修工程有關。"]))

Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 181.22it/s]

[['法國', '總統', '馬克宏', '已', '到', '現場', '勘災', '，', '初步', '傳出', '火警', '可能', '與', '目前', '聖母院', '的', '維修', '工程', '有關', '。']]


In [19]:
pred = bert_segment(["".join(sent) for sent in raw_test])
actual = raw_test
print(actual[0])
print(pred[0])
print(score(actual, pred))

Running Prediction: 100%|██████████| 1765/1765 [00:31<00:00, 56.06it/s]


['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風', '舞班', '，']
(0.9359889995416476, 0.9363557541601361, 0.9361723409309711)


In [20]:
text = """勞資評斷委員會，昨日下午假社會局會議室舉行第八次會議。出席吳開先，潘公展，張維，杜月笙（朱文德代），宣鐵吾（黃東昇代），水祥雲，黃仁貴，佘敬成，吳蘊初（田和卿代）。列席李劍華，劉靖　，樊振邦等。因生活指數凍結後，二月份實行差額貼補制度，規定每名貼補七萬三千元。各工廠公會甚為關懷，紛紛請求社會局解釋。惟因無統一辦法，恐以後引起糾紛，故商討決定加工等發給差金辦法，細則為：（一）件工，無論工人生產量之多寡，概以工作日數作為發給差金之標準。（二）童工（包括學徒藝徒養成工等，）凡已免費供給膳食者，其米油鹽煤之差金不予發給，祗發給布糖兩項。凡不供膳者，給予三分之一差金。（三）職工請請假工資照扣者，其差金比例照扣。但病假不扣差金。此外，習慣上向不扣除工資者，差金亦不扣除。（四）各工廠公司行號解雇職工，依照評斷會辦法發給遺散費者，應自二月份起將差金併入工資內發給之。（五）延長工作時間以及臨時加工，不再發給差金。（六）各廠過去免費發給食米者，差金中食米一項不予扣除。（七）工資多寡差金不加區別。另西服時裝等業工人在旺月淡月及停工等情形下，差金應如何發給，尚待研究。"""
gold = """勞資 評斷 委員會 ， 昨日 下午 假 社會局 會議室 舉行 第八 次 會議 。 
出席 吳開先 ， 潘公展 ， 張維 ， 杜月笙（朱文德代）， 宣鐵吾（黃東昇代） ， 水祥雲 ， 黃仁貴 ， 佘敬成 ， 吳蘊初（田和卿代） 。
列席 李劍華 ， 劉靖 ， 樊振邦 等 。 
因 生活 指數 凍結 後 ， 二月份 實行差額 貼補 制度 ， 規定 每 名 貼補 七萬三千元 。 
各 工廠 公會 甚 為 關懷 ， 紛紛 請求 社會局 解釋 。 
惟因 無 統一 辦法 ， 恐 以後 引起 糾紛 ， 故 商討 決定 加工 等 發給 差金 辦法 ， 細則 為 ：
（ 一 ）件工，無論 工人 生產量 之 多寡 ， 概 以 工作 日數 作 為 發給 差金 之標準 。 
（ 二 ）童工（ 包括 學徒 藝徒 養成工 等 ， ）凡 已 免費 供給 膳食 者 ， 其 米 油 鹽 煤 之 差金 不予 發給 ， 
祗 發給 布 糖 兩項 。
凡 不 供膳者 ， 給予 三分之一 差金 。
（ 三 ）職工 請 請假 工資 照扣 者 ， 其 差金 比例照扣 。
但 病假 不 扣 差金 。
此外 ， 習慣 上 向 不 扣除 工資 者 ， 差金 亦 不 扣除 。
（ 四 ）各 工廠 公司 行號 解雇 職工 ， 依照 評斷會 辦法 發給 遺散費 者 ， 應 自 二月份 起 將 差金 併入 工資 內 發給 之 。
（ 五 ）延長 工作 時間 以及 臨時 加工 ， 不再 發給 差金 。
（ 六 ）各 廠 過去 免費 發給 食米 者 ， 差金 中 食米 一項 不予 扣除 。
（ 七 ）工資 多寡 差金 不加 區別 。
另 西服 時裝 等 業 工人 在 旺月 淡月 及 停工 等 情形 下 ， 差金 應 如何 發給 ，尚 待 研究。"""


text = "寗紹商輪公司於二十四日午後二時半在寗波旅滬同鄕會開第十八屆股東年會到會股東計一萬二千二百七十七股計九千一百二十一權方椒伯主席袁經理報告丙寅年營業狀况監察人謝蓮卿報告丙寅年帳略次股東票選董事監察人開票結果當留舊董事三人方椒伯·方樵苓·樂振葆·新選董事八人何謀軒·謝蘅牕·孫梅堂·王心貫·樓恂如·何積璠·傅品圭·徐棣蓀·候補董事十一人袁履登·屠康侯·項松茂·洪賢鈁·洪賢賓·何紹庭·李志方·謝蓮卿·王雲甫·施才高·李詠裳監察人五人石運乾·謝蓮卿·洪賢鈁·周仰山·洪雁賓·候補監察人五人王雲甫·傅其霖·樓恂如·劉鴻生·項松茂旋即散會"
text = "七月初一日福建正主考葛寳華副主考黄紹第今晨過蘇赴閩太倉州蔣禀到同知羅承恩禀謝奉委閔行釐局差卽辭並銷發審局差新選建始縣逃後辭行赴湖北代理南匯縣楊十晟禀辭赴任知縣吳俟卿禀銷南路總巡差又錢國選禀知到南路總巡差　徐樹森禀知赴寗見制憲卽辭巡檢沈廷楨禀銷吳淞江釐局司員差典史薛觀禀知奉委盤門外帮同彈壟差○初三日同知銜湖北候補知縣徐元瀛浙江人由鄂來通判陳寳樽謝委盛澤厘局差委赴四川守提償欵委員候補知縣鹿　理禀辭赴川圓　門外二叚巡查委員候補縣丞竇以　禀知代辦城外陸路總巡差道庫大使許光熙謝委荆溪縣保甲差卽辭主簿郝炳麟禀知奉委文闈執事差巡檢何天綬謝委　門旱關差"
text = "新聞紙之流布於寰區也香港則間日呈奇峙佳名於三秀滬瀆則每晨抽秘鬥彩筆於兩家或著錄應來復之期教堂握槧或成帙在合朔之候京國傳書凡此紀事而纂言莫不標新而領異議分鄉校願考見夫濟世之經猷源討楹書願輯夫證今之學問下至方言里語足備談諧雜筆小詩亦供嗢噱奈日力之有限致篇幅之無多花類折枝僅悦一時之目玉非全壁誰知千古之心斷爛之朝錄堪聞見之錄難遍用特勤加搜討遍訪知交積三十日之斷錦零縑居然成幅合四大洲之隋珠和璧用示奇珍擬為瀛寰瑣紀一書凡巳登申報者不錄每月以朔日出書一卷其價則每本八十文若本館躉售則每本六十五文初印之時以二千本為率如有各書坊定印必當預先知會臨時不能議添矣備中朝之史料名敢託夫官廣異域之談資陋不嫌夫蠻語瑣聞兼述田附搜神志怪之餘碎事同登不薄巷議街談之末所願文壇健者儒林丈人惠贈瑶章共襄盛舉庶幾琳瑯日耀如入寶山梨棗風行不慚詞苑則本館有厚望焉"
pred = bert_segment([text])
actual = [gold.replace("\n", " ").split(" ")]
print(" ".join(pred[0]))
#print(actual[0])
#print(score(actual, pred))


Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 173.86it/s]

新聞紙 之 流布 於 寰區 也 香港 則 間 日呈 奇峙 佳名 於 三秀 滬 瀆 則 每 晨 抽 秘鬥 彩筆 於 兩 家 或 著錄 應 來復 之 期 教堂 握槧 或 成帙 在 合朔 之 候 京國 傳書 凡 此 紀事 而 纂言 莫不 標新 而 領 異議 分 鄉校 願 考見 夫 濟世 之 經猷 源 討 楹書 願 輯 夫 證 今 之 學問 下 至 方言 里語 足 備 談 諧 雜筆 小詩 亦 供 嗢 噱 奈日力 之 有限 致 篇幅
